# AMSA: Ensemble Random Forest Classification

## Relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold # for creating k-fold cv and deal with class imbalanceissue

from sklearn.ensemble import RandomForestClassifier # for the random forest classification model

from sklearn.compose import make_column_transformer  # for applying appropriate transformations for each columns
from category_encoders.cat_boost import CatBoostEncoder # for Encoding categorical variables in a way that makes tree-based method able to efficiently evaluate splits

from sklearn.pipeline import Pipeline # for generating pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # for tuning the hyperparameters

/Users/hiimpian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Data

In [2]:
default_train_complete = pd.read_csv('../data/default_train_complete.csv').drop(columns = ['LoanID'])
X_train = default_train_complete.drop(columns = ['Default'])
y_train = pd.DataFrame(default_train_complete['Default'])

In [4]:
default_train_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41514 entries, 0 to 41513
Data columns (total 47 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age                                   41514 non-null  int64  
 1   Income                                41514 non-null  int64  
 2   LoanAmount                            41514 non-null  int64  
 3   CreditScore                           41514 non-null  int64  
 4   MonthsEmployed                        41514 non-null  int64  
 5   NumCreditLines                        41514 non-null  int64  
 6   InterestRate                          41514 non-null  float64
 7   LoanTerm                              41514 non-null  int64  
 8   DTIRatio                              41514 non-null  float64
 9   Education                             41514 non-null  object 
 10  EmploymentType                        41514 non-null  object 
 11  MaritalStatus  

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41514 entries, 0 to 41513
Data columns (total 46 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age                                   41514 non-null  int64  
 1   Income                                41514 non-null  int64  
 2   LoanAmount                            41514 non-null  int64  
 3   CreditScore                           41514 non-null  int64  
 4   MonthsEmployed                        41514 non-null  int64  
 5   NumCreditLines                        41514 non-null  int64  
 6   InterestRate                          41514 non-null  float64
 7   LoanTerm                              41514 non-null  int64  
 8   DTIRatio                              41514 non-null  float64
 9   Education                             41514 non-null  object 
 10  EmploymentType                        41514 non-null  object 
 11  MaritalStatus  

In [6]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41514 entries, 0 to 41513
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Default  41514 non-null  int64
dtypes: int64(1)
memory usage: 324.5 KB


## Cross validation

In [7]:
cv_folds = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 30897) # the shuffle argument is used to randomize the order of the elements in the data before splitting them into folds

## Assessment metrics

In [8]:
metrics_set = {
    'recall': 'recall', 
    'bal_accuracy': 'balanced_accuracy', 
    'precision': 'precision', 
    'f1': 'f1', 
    'roc_auc': 'roc_auc'
}

## Modelling

When we are implementing random forest, we need to keep in mind that it is a tree-based method. Which means that the same preprocessing pipeline won't be the same as for k-NN. More specifically, with tree-based methods, they are insensitive to feature scaling. In other words, the results of the splits don't change with any monotonic transformation to the features. Consequently, we won't be applying the "step_scaling" procedure like when we did it for the k-NN model. 

In addition, when considering the fact that we are now dealing with tree-based method, we applying OneHotEncoder will increase the number of features in the current feature space. This can be problematic for tree-based method because with one hot encoding, the model will have to work with sparse matrix (features with a lot of 0 values) especially for features that consists of many levels (unique values), suggesting that these tree-based methods won't be able to efficiently evaluate the splits of each features. Notably, when the data is sparse, there can be many uninformative splits and a tree structure that does not efficiently capture the underlying pattern in the data. 

One possible soluion would be to use CatBoostEncoder()

Specifying the model

In [9]:
rf_class_model = RandomForestClassifier(random_state = 12345, n_jobs = -1) # specifying the parameter n_jobs to be -1 to tell the machine to use all the available cpu cores when doing the computations, essentially speeding up the process

Specifying preprocessing transformations

In [10]:
preprocessor = make_column_transformer(
    (CatBoostEncoder(), [str(col) for col in X_train.select_dtypes(['object', 'category'])]), 
    remainder = 'passthrough'
)

Specifying the model's main pipeline

In [11]:
rf_class_pipeline = Pipeline(
    steps = [
        ('preprocessor', preprocessor), 
        ('rf_class_model', rf_class_model)  
    ]
)

Setting up the tuning grid

In [13]:
rf_class_tune_grid = {
    'rf_class_model__n_estimators': [int(x) for x in range(800, 2001, 100)], # number of trees in the forest
    'rf_class_model__max_features': ['log2', 'sqrt'], 
    'rf_class_model__max_depth': [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)], # the maximum number of levels (or layers) in tree, more layers more prone to overfittig, so limit up to 10 should be enough
    'rf_class_model__min_samples_split': [3, 5, 7] # the minimum number of samples required to split a node at each child nodes after the split
}
rf_class_tune_grid

{'rf_class_model__n_estimators': [800,
  900,
  1000,
  1100,
  1200,
  1300,
  1400,
  1500,
  1600,
  1700,
  1800,
  1900,
  2000],
 'rf_class_model__max_features': ['log2', 'sqrt'],
 'rf_class_model__max_depth': [2, 4, 6, 8, 10],
 'rf_class_model__min_samples_split': [3, 5, 7]}

Initiating the grid search object

In [11]:
rf_class_grid_search = GridSearchCV(
    estimator = rf_class_pipeline,
    param_grid = rf_class_tune_grid, 
    scoring = metrics_set,
    refit = 'recall', 
    cv = cv_folds, 
    error_score = 'raise'
)

Fitting the grid search to tune the hyperparameters

In [ ]:
rf_class_grid_search.fit(X = X_train, y = y_train)

Due to computational power issue, using RandomizedSearchCV() can significantly speed up the process as it instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. In doing so, you can try much more diverse combinations of hyperparameters at a much lower computational cost.

In [14]:
rf_class_randomized_grid_search = RandomizedSearchCV(
    estimator = rf_class_pipeline, 
    param_distributions = rf_class_tune_grid, 
    scoring = metrics_set, 
    refit = 'recall',
    cv = cv_folds, 
    error_score = 'raise'
)

Fitting the randomized grid search to tune the hyperaparameters

In [ ]:
rf_class_randomized_grid_search.fit(X = X_train, y = y_train)

Obtaining the results of the hyperparameter tuning

In [15]:
rf_class_tune_results = pd.DataFrame(rf_class_randomized_grid_search.cv_results_)
rf_class_tune_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf_class_model__n_estimators,param_rf_class_model__min_samples_split,param_rf_class_model__max_features,param_rf_class_model__max_depth,params,split0_test_recall,...,split3_test_roc_auc,split4_test_roc_auc,split5_test_roc_auc,split6_test_roc_auc,split7_test_roc_auc,split8_test_roc_auc,split9_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc
0,22.892910,2.710403,1.086714,0.332076,1750,5,sqrt,2,"{'rf_class_model__n_estimators': 1750, 'rf_cla...",0.612526,...,0.719070,0.718483,0.718633,0.703244,0.719097,0.721248,0.724796,0.718582,0.009353,8
1,38.404634,4.062490,1.090308,0.677046,1250,5,log2,8,"{'rf_class_model__n_estimators': 1250, 'rf_cla...",0.655884,...,0.736684,0.737227,0.742819,0.729184,0.735298,0.743596,0.736321,0.738274,0.008237,2
2,20.195181,1.399348,0.406401,0.075647,700,5,sqrt,6,"{'rf_class_model__n_estimators': 700, 'rf_clas...",0.639367,...,0.734153,0.736006,0.736853,0.722894,0.732292,0.739975,0.735159,0.734874,0.008540,4
3,3.607584,0.365984,0.229505,0.053208,350,5,log2,2,"{'rf_class_model__n_estimators': 350, 'rf_clas...",0.611149,...,0.713253,0.711968,0.713809,0.698951,0.714001,0.715694,0.719594,0.713588,0.009741,10
4,28.646488,2.065320,0.904057,0.366217,1100,5,sqrt,6,"{'rf_class_model__n_estimators': 1100, 'rf_cla...",0.638679,...,0.734214,0.736114,0.737236,0.723256,0.732223,0.740271,0.734781,0.734962,0.008530,3
5,27.504127,2.599172,0.659847,0.171005,800,3,log2,10,"{'rf_class_model__n_estimators': 800, 'rf_clas...",0.660702,...,0.738379,0.740705,0.745794,0.731855,0.735150,0.746021,0.737615,0.740249,0.008065,1
6,11.866467,1.810130,0.534279,0.120795,1050,5,sqrt,2,"{'rf_class_model__n_estimators': 1050, 'rf_cla...",0.615279,...,0.720828,0.719234,0.720210,0.704100,0.719453,0.721882,0.725254,0.719517,0.009414,7
7,15.233442,0.867003,0.471558,0.036339,950,3,log2,4,"{'rf_class_model__n_estimators': 950, 'rf_clas...",0.628355,...,0.725314,0.725969,0.728782,0.712809,0.725440,0.729561,0.729020,0.726423,0.009025,5
8,13.451880,0.334077,0.640535,0.026017,1500,3,log2,2,"{'rf_class_model__n_estimators': 1500, 'rf_cla...",0.608396,...,0.717443,0.715936,0.717821,0.700951,0.717680,0.719497,0.722952,0.716981,0.009421,9
9,12.246643,0.278657,0.510715,0.026267,1200,5,sqrt,2,"{'rf_class_model__n_estimators': 1200, 'rf_cla...",0.616655,...,0.720499,0.719644,0.720168,0.704587,0.719808,0.722256,0.725003,0.719628,0.009325,6


In [16]:
rf_class_tune_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 74 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   mean_fit_time                            10 non-null     float64
 1   std_fit_time                             10 non-null     float64
 2   mean_score_time                          10 non-null     float64
 3   std_score_time                           10 non-null     float64
 4   param_rf_class_model__n_estimators       10 non-null     object 
 5   param_rf_class_model__min_samples_split  10 non-null     object 
 6   param_rf_class_model__max_features       10 non-null     object 
 7   param_rf_class_model__max_depth          10 non-null     object 
 8   params                                   10 non-null     object 
 9   split0_test_recall                       10 non-null     float64
 10  split1_test_recall                       10 non-null 

In [ ]:
# fig, axes = plt.subplots(2, 2, figsize = (10, 10))

# axes[0,0].plot(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_recall'])
# axes[0,0].scatter(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_recall'])
# axes[0,0].set_title('sensitivity')
# axes[0,0].set_ylabel('mean')
# axes[0,0].set_xlabel('max_depth')
# axes[0,0].grid()

# axes[0,1].plot(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_bal_accuracy'])
# axes[0,1].scatter(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_bal_accuracy'])
# axes[0,1].set_title('balanced accuracy')
# axes[0,1].set_ylabel('mean')
# axes[0,1].set_xlabel('max_depth')
# axes[0,1].grid()

# axes[1,0].plot(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_precision'])
# axes[1,0].scatter(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_precision'])
# axes[1,0].set_title('precision')
# axes[1,0].set_ylabel('mean')
# axes[1,0].set_xlabel('max_depth')
# axes[1,0].grid()

# axes[1,1].plot(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_f1'])
# axes[1,1].scatter(rf_class_tune_results.loc[:, 'param_rf_class_model__max_depth'], rf_class_tune_results.loc[:, 'mean_test_f1'])
# axes[1,1].set_title('f1')
# axes[1,1].set_ylabel('mean')
# axes[1,1].set_xlabel('max_depth')
# axes[1,1].grid()

In [17]:
# First, create a data frame that contains the number of neighbors, the mean test estimates and the standard error of each mean test estimates. 
n_cv = 10
n_estimators = []
max_features = []
max_depth = []
min_samples_split = []
mean_estimates = []
std_err = []

for i in range(len(rf_class_tune_results)):
    n_estimators.append(rf_class_tune_results.loc[i, 'param_rf_class_model__n_estimators'])
    max_features.append(rf_class_tune_results.loc[i, 'param_rf_class_model__max_features'])
    max_depth.append(rf_class_tune_results.loc[i, 'param_rf_class_model__max_depth'])
    min_samples_split.append(rf_class_tune_results.loc[i, 'param_rf_class_model__min_samples_split'])
    mean_estimates.append(rf_class_tune_results.loc[i, 'mean_test_recall'])
    std_err_i = rf_class_tune_results.loc[i, 'std_test_recall']/np.sqrt(n_cv)
    std_err.append(std_err_i)

rf_class_tune_results_cleaned = pd.DataFrame({
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth, 
    'min_samples_split': min_samples_split, 
    "mean_test_recall": mean_estimates, 
    "std_err_test_recall": std_err
})

rf_class_tune_results_cleaned = rf_class_tune_results_cleaned.sort_values(by = "mean_test_recall", ascending = False)

rf_class_tune_results_cleaned

,n_estimators,max_features,max_depth,min_samples_split,mean_test_recall,std_err_test_recall
5,800,log2,10,3,0.668135,0.005159
1,1250,log2,8,5,0.662010,0.005454
4,1100,sqrt,6,5,0.655471,0.006080
2,700,sqrt,6,5,0.655196,0.005712
7,950,log2,4,3,0.642326,0.005539
9,1200,sqrt,2,5,0.631246,0.005096
6,1050,sqrt,2,5,0.631039,0.004954
8,1500,log2,2,3,0.628630,0.005284
0,1750,sqrt,2,5,0.628286,0.005459
3,350,log2,2,5,0.628217,0.005091


In [18]:
best_estimator = rf_class_tune_results_cleaned.iloc[0, :] # becareful to USE iloc not loc to access the index of the best estimator not the value of the index!
#print(best_estimator["mean_test_recall"])
best_estimator_range = best_estimator['mean_test_recall'] - best_estimator['std_err_test_recall']
#print(best_estimator_range)

best_estimator_one_std_err_list = []
for i in range(len(rf_class_tune_results_cleaned)):
    if rf_class_tune_results_cleaned.loc[i, 'mean_test_recall'] >= best_estimator_range:
        if rf_class_tune_results_cleaned.loc[i, 'max_depth'] < best_estimator['max_depth'] or rf_class_tune_results_cleaned.loc[i, 'min_samples_split'] < best_estimator['min_samples_split']: 
            best_estimator_one_std_err_list.append(rf_class_tune_results_cleaned.loc[i, :])
    else:
        best_estimator_one_std_err_list.append(best_estimator)
            
best_estimator_one_std_err_df = pd.DataFrame(best_estimator_one_std_err_list)

best_estimator_one_std_err_df

rf_best_estimator_one_std_err = best_estimator_one_std_err_df.tail(1)
rf_best_estimator_one_std_err

,n_estimators,max_features,max_depth,min_samples_split,mean_test_recall,std_err_test_recall
5,800,log2,10,3,0.668135,0.005159


Based on the result above, it is evident that the best hyperparameter when applying the one standard error rule is n_estimators of 1100, max_features is the square root of the number of features, max_depth of 10, and min_samples_split of 5. Which yield sensitivity rate of 67.1232%. 

Finalize the workflow by specifying the model with the tuned hyperparameters

In [19]:
rf_class_best_estimator = RandomForestClassifier(
    n_estimators = rf_best_estimator_one_std_err['n_estimators'].iloc[0], 
    max_features = rf_best_estimator_one_std_err['max_features'].iloc[0], 
    max_depth = rf_best_estimator_one_std_err['max_depth'].iloc[0],
    min_samples_split = rf_best_estimator_one_std_err['min_samples_split'].iloc[0]
)

rf_class_best_estimator

RandomForestClassifier(max_depth=10, max_features='log2', min_samples_split=3,
                       n_estimators=800)

In [20]:
rf_class_tune_results[rf_class_tune_results['param_rf_class_model__n_estimators'] == 800 and rf_class_tune_results['param_rf_class_model__max_depth'] == 10 and rf_class_tune_results['param_rf_class_model__max_features'] == 'log2' and rf_class_tune_results['param_rf_class_model__min_samples_split'] == 3]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().